In [0]:
import pandas as pd
import datetime
from datetime import timedelta
import time
import json
import pytz


from pyspark.sql.types import StructType, ArrayType  
from pyspark.sql.functions import col, when
from delta.tables import *
from pyspark.sql.functions import *
import numpy as np
# from azure.cosmos import CosmosClient

In [0]:
# 储存账户名称
storage_account_name = ""
container_name=""
storage_account_access_key = ""
connect_str = ""

file_location = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/"
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

In [0]:
day = []
file_location = 
elevator_list = ['F8N94277/','F8N94278/','D7SE7336/']

# 前一天
for day_num in range(1,3):
  date = datetime.datetime.now(pytz.timezone('Asia/Shanghai')) - timedelta(days=day_num)
  day.append(str(date.year) +'/' + str(date.month) +'/' + str(date.day))
print(day)

['2021/5/13', '2021/5/12']

In [0]:
final = pd.DataFrame()
final2 = pd.DataFrame()
for elevator in elevator_list:
  new_file_location = file_location + elevator
  pic_list = []
  for date in day:
    pic = dbutils.fs.ls(new_file_location + '/' +date)
    for i in pic:
      i = int(i[1].replace('.json',''))
      pic_list.append(i)
  pic_list = sorted(pic_list)
  df = pd.DataFrame(pic_list,columns = ['end_time'])
  df['start_time'] = df.shift(1)
  df['gap'] = df['end_time'] - df['start_time']
  df.dropna(axis=0,how='any',inplace=True)
  df['elevator'] = elevator
  df = df.sort_values(by = 'gap',ascending=False)
#   df['end_time'] = pd.to_datetime(df['end_time'],unit='ms',origin=pd.Timestamp('1970-01-01 08:00:00'))
#   df['start_time'] = pd.to_datetime(df['start_time'],unit='ms',origin=pd.Timestamp('1970-01-01 08:00:00'))
  final = pd.concat([final,df],axis=0)
  
  
  value_1_12 , value_13_15 , value_16_20 ,value_21_30 , value_31= 0 ,0,0,0,0
  for value in df['gap']:
    if value <= 12:
      value_1_12 += 1
    if 12< value <=15:
      value_13_15 += 1
    if 15< value <=20:
      value_16_20 += 1
    if 20 < value <=30:
      value_21_30 += 1
    if value >30:
      value_31 += 1
  df2 =[elevator,value_1_12 , value_13_15 , value_16_20 ,value_21_30 , value_31]
  df2 = pd.DataFrame(df2,index = ['电梯','1-12','13-15','16-20','21-30','大于31']).T
  final2 = pd.concat([final2,df2],axis=0)

final.set_index('elevator',inplace=True)
final2.set_index('电梯',inplace=True)
final.to_csv('/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/Upload_Time/Unix/%s——%s.csv' % (day[0].replace('/','-'),day[1].replace('/','-')))
final2.to_csv('/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/Upload_Time/Result/%s——%s.csv' % (day[0].replace('/','-'),day[1].replace('/','-')))
print('储存完成：' + '/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/Upload_Time/Unix/%s——%s.csv' % (day[0].replace('/','-'),day[1].replace('/','-')))
print('储存完成：' + '/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/Upload_Time/Result/%s——%s.csv' % (day[0].replace('/','-'),day[1].replace('/','-')))